In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('restaurant-comments.xlsx')

In [4]:
df.head()

,comments,date
0,这辈子最爱吃的火锅，一星期必吃一次啊！最近才知道他家还有免费鸡蛋羹………………炒鸡好吃炒鸡嫩...,2017-05-14 16:00:00
1,第N次来了，还是喜欢?……<br><br>从还没上A餐厅的楼梯开始，服务员已经在那迎宾了，然...,2017-05-10 16:00:00
2,大姨过生日，姐姐定的这家A餐厅的包间，服务真的是没得说，A餐厅的服务也是让我由衷的欣赏，很久...,2017-04-20 16:00:00
3,A餐厅的服务哪家店都一样，体贴入微。这家店是我吃过的排队最短的一家，当然也介于工作日且比较晚...,2017-04-25 16:00:00
4,因为下午要去天津站接人，然后我倩前几天就说想吃A餐厅，然后正好这有，就来这吃了。<br>来的...,2017-05-21 16:00:00


注意这里的时间列。如果你的Excel文件里的时间格式跟此处一样，包含了__日期和时间(注意二者缺一不可)__ ，那么Pandas会非常智能地帮你把它识别为时间格式，接着往下做就可以了。
反之，如果你获取到的时间只精确到日期，例如"2017-04-20"这样，那么Pandas只会把它当做字符串，后面的时间序列分析无法使用字符串数据。解决办法是在这里加入以下两行代码：

__from dateutil import parser
df['date'] = df.date.apply(parser.parse)__

In [7]:
text = df.comments.iloc[0]

In [8]:
# 调用SnowNLP情感分析工具
from snownlp import SnowNLP
s = SnowNLP(text)

In [9]:
s.sentiments

0.6331975099099649